# Preferential Bayesian Optimization: Predictive Entropy Search
This notebook demonstrates the use of the Predictive Entropy Search (PES) acquisition function on ordinal (preference) data, and compares its performance directly to the Dueling-Thompson sampling acquisition function by Gonzalez et al (2017). 

We conduct experiments over simple 1-dimensional values in the domain [0, 1] and define the following preference ordering over them: given the Forrester function f (1-dimensional test function by Forrester et al. (2008)), and two inputs a and b, a > b if f(a) < f(b).

Formulation by Nguyen Quoc Phong.

In [ ]:
import numpy as np
import gpflow
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import os

from gpflow.utilities import set_trainable, print_summary
gpflow.config.set_default_summary_fmt("notebook")

In [ ]:
sys.path.append(os.path.split(os.path.split(os.getcwd())[0])[0]) # Move 2 levels up directory to import PBO
import PBO

In [ ]:
def forrester(x):
    """
    1-dimensional test function by Forrester et al. (2008)
    Defined as f(x) = (6x-2)^2 * sin(12x-4)
    :param x: float in domain [0, 1]
    """
    return (6*x-2)*(6*x-2)*np.sin(12*x-4)

def forrester_get_y(X):
    """
    Returns np array of shape (num_data, 1), indicating the input value with the most preferred Forrester value
    (lower is better)
    param X: np array of shape (num_data, num_choices, 1)
    """
    
    forr_vals = np.argmin(forrester(X), axis=1)
    return np.squeeze(np.take_along_axis(X, np.expand_dims(forr_vals, axis=2), axis=1), axis=2)

In [ ]:
def plot_gp(model, X, y, title):
    #Plotting code from GPflow authors

    ## generate test points for prediction
    xx = np.linspace(-0.1, 1.1, 100).reshape(100, 1)  # test points must be of shape (N, D)

    ## predict mean and variance of latent GP at test points
    mean, var = model.predict_f(xx)

    ## generate 10 samples from posterior
    samples = model.predict_f_samples(xx, 10)  # shape (10, 100, 1)

    ## plot 
    plt.figure(figsize=(12, 6))
    plt.plot(X, y, 'kx', mew=2)
    plt.plot(xx, mean, 'C0', lw=2)
    plt.fill_between(xx[:,0],
                     mean[:,0] - 1.96 * np.sqrt(var[:,0]),
                     mean[:,0] + 1.96 * np.sqrt(var[:,0]),
                     color='C0', alpha=0.2)

    plt.plot(xx, samples[:, :, 0].numpy().T, 'C0', linewidth=.5)
    plt.xlim(-0.1, 1.1)
    plt.title(title)

## 1.  Learning from ordinal data

In [ ]:
# Sample data
X = np.array([[[0.2], [0.4]],
              [[0.4], [0.7]],
              [[0.76], [0.9]],
              [[0.9], [0.2]],
              [[0.7], [0.9]],
              [[0.2], [0.7]],
              [[0.7], [0.76]]])
y = forrester_get_y(X)

Learn variational parameters for each input seen in training data:

In [ ]:
q_mu, q_var, inputs = PBO.models.learning.train_model(X, y, num_steps=3000)

Create sparse variational Gaussian process model using these parameters:

In [ ]:
kernel = gpflow.kernels.RBF()
likelihood = gpflow.likelihoods.Gaussian()
model = PBO.models.learning.init_SVGP(q_mu, q_var, inputs, kernel, likelihood)

In [ ]:
# Tune hyperparameters. TODO: Find automatic way to do this
model.kernel.lengthscale.assign(0.07)

In [ ]:
print_summary(model)

Visualize the model so far:

In [ ]:
func_evals = np.expand_dims(q_mu.numpy(),1)

In [ ]:
plot_gp(model, inputs, func_evals, "GP")

## 2. Predictive Entropy Search

In [ ]:
num_runs = 10
num_evals = 20
num_samples = 100
num_choices = 2
input_dims = 1
num_maximizers = 20
num_init_points = 5

The following function captures what was done in the previous section, as we need to retrain the model after every evaluation.

In [ ]:
def train_and_visualize(X, y, title):
    
    # Train model with data
    q_mu, q_var, inputs = PBO.models.learning.train_model(X, y, num_steps=3000)
    kernel = gpflow.kernels.RBF()
    likelihood = gpflow.likelihoods.Gaussian()
    model = PBO.models.learning.init_SVGP(q_mu, q_var, inputs, kernel, likelihood)
    model.kernel.lengthscale.assign(0.07)
    func_evals = np.expand_dims(q_mu.numpy(),1)
    
    # Visualize model
    plot_gp(model, inputs, func_evals, title)
    
    return model, inputs, func_evals

This function is our main metric for the performance of the acquisition function: The closer the model's best guess to the target (in this case, the global minimum of the Forrester function), the better.

In [ ]:
def best_guess(model):
    """
    Returns a GP model's best guess of the global maximum of f.
    """
    xx = np.linspace(0.0, 1.0, 10000000).reshape(10000000, 1)
    res = model.predict_f(xx)[0].numpy()
    return xx[np.argmax(res)]

Store the results in these arrays:

In [ ]:
num_data_at_end = int((num_init_points-1) * num_init_points / 2 + num_evals)
X_results = np.zeros([num_runs, num_data_at_end, num_choices, input_dims])
y_results = np.zeros([num_runs, num_data_at_end, 1])
best_guess_results = np.zeros([num_runs, num_evals, input_dims])

Create the initial values for each run:

In [ ]:
np.random.seed(0)
init_points = np.random.uniform(size=[num_runs, num_init_points, input_dims])
num_combs = int((num_init_points-1) * num_init_points / 2)
init_vals = np.zeros([num_runs, num_combs, num_choices, input_dims])
for run in range(num_runs):
    cur_idx = 0
    for init_point in range(num_init_points-1):
        for next_point in range(init_point+1, num_init_points):
            init_vals[run, cur_idx, 0] = init_points[run, init_point]
            init_vals[run, cur_idx, 1] = init_points[run, next_point]
            cur_idx += 1

The following loops carry out the Bayesian optimization algorithm over a number of runs, with a fixed number of evaluations per run.

In [ ]:
for run in range(num_runs):
    print("Beginning run %s" % (run))
    
    X = init_vals[run]
    y = forrester_get_y(X)
    
    model, inputs, func_evals = train_and_visualize(X, y, "Run {}: Initial model".format(run))

    for evaluation in range(num_evals):
        print("Beginning evaluation %s" % (evaluation)) 

        # Sample possible next queries
        
        samples = PBO.acquisitions.pes.sample_inputs(inputs, num_samples, num_choices)

        # Sample maximizers
        print("Evaluation %s: Sampling maximizers" % (evaluation))
        maximizers = PBO.acquisitions.pes.sample_maximizers(X=inputs, 
                                                            y=func_evals, 
                                                            count=num_maximizers,
                                                            D=1000,
                                                            variance=1.0)

        # Calculate PES value I for each possible next query
        print("Evaluation %s: Calculating I" % (evaluation))
        I_vals = PBO.acquisitions.pes.I_batch(samples, maximizers, model)

        # Select query that maximizes I
        next_idx = np.argmax(I_vals)
        next_query = samples[next_idx]
        print("Evaluation %s: Next query is %s with I value of %s" % (evaluation, next_query, I_vals[next_idx]))

        X = np.concatenate([X, [next_query]])
        # Evaluate objective function
        y = forrester_get_y(X)
        
        print("Evaluation %s: Training model" % (evaluation))
        model, inputs, func_evals = train_and_visualize(X, y, "Run {}: Evaluation {}".format(run, evaluation))

        best_guess_results[run, evaluation, :] = best_guess(model)

    X_results[run] = X
    y_results[run] = y

In [ ]:
k = gpflow.kernels.RBF()

In [ ]:
print_summary(k)

In [ ]:
gpflow.kullback_leiblers.gauss_kl

In [ ]:
n = inputs.shape[0]

In [ ]:
q_mu = tf.Variable(np.zeros([n, 1]), name="q_mu", dtype=tf.float64)
q_sqrt_latent = tf.Variable(np.expand_dims(np.eye(n), axis=0), name="q_var", dtype=tf.float64)
q_sqrt = tf.linalg.band_part(q_sqrt_latent, -1, 0) # Force lower triangular

In [ ]:
q_sqrt

In [ ]:
cov_mat = k.K(inputs)

In [ ]:
cov_mat

In [ ]:
gpflow.kullback_leiblers.gauss_kl(q_mu, q_sqrt, cov_mat)

In [ ]:
tf.linalg.diag_part(cov_mat)

In [ ]:
inputs.shape

In [ ]:
k.trainable_variables

In [ ]:
lst = [1,2]

In [ ]:
lst + list(k.trainable_variables)

In [ ]:
print_summary(k)

In [ ]:
D_idxs